In [1]:
import numpy as np
import sys
sys.path.append("../../HelperFiles")
from helper import *
from shapley_sampling2 import *
from train_models import *
from load_data import *

import warnings
warnings.filterwarnings('ignore')

/Users/jeremygoldwasser/opt/anaconda3/envs/shap/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load dataset and train model

In [2]:
X_train, y_train, X_test, y_test, mapping_dict = load_data("census", "../Data")
xloc = X_test[0:1]
# model, approx, true_shap_vals = train_logreg(X_train, y_train, xloc, mapping_dict)
model, approx, true_shap_vals = train_neural_net(X_train, y_train, xloc, mapping_dict)

print("Class imbalance: {}%".format(round(100*(max(np.mean(y_test), 1-np.mean(y_test))))))
Y_preds = (model(X_test) > 0.5).astype("int")
print("NN {}% accuracy".format(round(np.mean(Y_preds == y_test)*100)))


KeyboardInterrupt: 

## Preliminary run of Shapley sampling

Not infrequently, expected number of samples to be significant exceeds max number of perms; sampling it, though, it's significant. This suggests our distribution is too conservative. Maybe we can remove the factor of 2.

In [4]:
# %run shapley_sampling2
np.random.seed(6)

shap_vals, diffs_all_feats = shapley_sampling_adaptive(model, X_train, xloc, K=1, alpha=0.2, mapping_dict=mapping_dict, multiplicative=True, max_n_perms=20000)

NameError: name 'model' is not defined

In [12]:
print(np.sum(shap_vals), model(xloc) - np.mean(model(X_train)))

print(np.sort(np.abs(shap_vals))[::-1])

0.587991810859778 0.66466135
[0.24051316 0.13317972 0.12231014 0.05500751 0.04828939 0.03875463
 0.01877386 0.01469499 0.00530414 0.00346821 0.00197111 0.0007789 ]


# Simulate Experimental FWER

## Model itself

Recommended number of samples often exceeds max # perms by a LOT, but then running the max # usually works. Makes me question whether the boundary is valid.

Using n_init=30 makes FWER more interesting, so that's what we show. Often n_init=50 makes the FWER 0. 

Still not sure one way or the other whether we need the factor of 2. Without, it's the Welch's t-test statistic. All these results are with the factor of 2.

results in 11/9 table

In [13]:
%run shapley_sampling2
# np.random.seed(2)
K = 1
top_K = []
for i in range(100):
    # print(i)
    shap_vals2, diffs_all_feats2 = shapley_sampling_adaptive(model, X_train, xloc, K=K, alpha=0.2, 
                            mapping_dict=mapping_dict, max_n_perms=10000, n_init=30, scale_var=True,
                            multiplicative=False)
    if shap_vals2 != "NA":
        est_top_K = get_ordering(shap_vals2)[:K]
        top_K.append(est_top_K)
        print(len(top_K))
        # if (i+1) % 5==0: 
        #     print(i+1)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [14]:
top_K = np.array(top_K)
most_common_row = mode_rows(top_K)
ct = 0
for idx in range(top_K.shape[0]):
    if np.array_equal(most_common_row, top_K[idx]):
        ct += 1
print(np.round(1 - ct / top_K.shape[0], 2)) # "FWER
print([len(diffs_all_feats2[j]) for j in range(len(diffs_all_feats2))])
print(top_K.shape[0], i)

0.0
[30, 30, 30, 92, 30, 30, 30, 30, 92, 30, 30, 30]
100 99


## Quadratic approximation
- GLM, Credit, BRCA, K=5: controls FWER
- NN, Credit, K=2: doesn't converge

In [ ]:
# %run shapley_sampling2
# np.random.seed(1)
# K = 5
# true_order = np.argsort(np.abs(true_shap_vals))[::-1]
# true_top_K = true_order[:K]
# same_top_K = []
# for i in range(100):
#     shap_vals2, diffs_all_feats2 = shapley_sampling_adaptive(approx, X_train, xloc, K=K,
#                                                 alpha=0.2, mapping_dict=mapping_dict)
#     est_top_K = np.argsort(np.abs(shap_vals2))[::-1][:K]
#     has_same_top_K = np.array_equal(true_top_K, est_top_K)
#     same_top_K.append(has_same_top_K)
#     if (i+1) % 5==0: print(i+1, np.round(1-np.mean(same_top_K),2)) # FWER
# np.round(1-np.mean(same_top_K),2) 